### Estructuración del dataset según categorías de Nolan

In [1]:
import pandas as pd

In [70]:
df_manifestos = pd.read_csv("../source/manifestos_etiquetados.csv")


In [71]:
df_manifestos = df_manifestos.drop('Unnamed: 0', axis = 1)

In [72]:
df_manifestos.head()

,Text,Code
0,DECLARACIÓN DE PRINCIPIOSFRENTE PARA LA VICTORIA,NaN
1,"Argentina, convicción y capacidad para constru...",NaN
2,La crisis ha sido el rostro de nuestro país du...,000
3,"Crisis económica, social, institucional, polít...",000
4,Sin reglas de juego claras y justas en lo econ...,503


In [109]:
# Diccionario de categorías con listas de códigos
categorias = {
    'Libertad Personal Alta': ['103.1', '201.1', '201.2', '202.3', '202.4', '503', '606.2', '604', '605.2', '607.1', '607.2', '607.3'],
    'Libertad Personal Baja': ['103.2', '202.2', '305.3', '601.2', '603', '604', '608.1', '608.3', '606.1', '605.1','504'],
    'Libertad Económica Alta': ['401', '402', '403', '404', '405', '407', '410', '411', '414'],
    'Libertad Económica Baja': ['406', '408', '409', '412', '413', '415', '416.1', '416.2', '503', '504']
}

# Convertir el diccionario en una lista de tuplas (Código, Categoría)
datos = [(codigo, categoria) for categoria, codigos in categorias.items() for codigo in codigos]

# Crear DataFrame
df_mapeo = pd.DataFrame(datos, columns=['Código', 'Categoría'])

# Inicializar DataFrame con todos los códigos únicos
codigos = set(categorias['Libertad Personal Alta'] + categorias['Libertad Personal Baja'] + 
              categorias['Libertad Económica Alta'] + categorias['Libertad Económica Baja'])

mapeo_libertad = pd.DataFrame(codigos, columns=['Código'])

# Asignar puntuaciones para libertad personal y económica
mapeo_libertad['libertad_personal_score'] = mapeo_libertad['Código'].apply(lambda x: 1 if x in categorias['Libertad Personal Alta'] else 
                                                           -1 if x in categorias['Libertad Personal Baja'] else 0)

mapeo_libertad['libertad_economica_score'] = mapeo_libertad['Código'].apply(lambda x: 1 if x in categorias['Libertad Económica Alta'] else 
                                                             -1 if x in categorias['Libertad Económica Baja'] else 0)

In [111]:
mapeo_libertad

,Código,libertad_personal_score,libertad_economica_score
0,607.2,1,0
1,606.1,-1,0
2,413,0,-1
3,407,0,1
4,601.2,-1,0
5,305.3,-1,0
6,201.1,1,0
7,605.2,1,0
8,402,0,1
9,202.4,1,0


In [112]:
# Unir el dataset original con el mapeo de libertad
dataset_mapeado = df_manifestos.merge(mapeo_libertad, left_on='Code', right_on='Código', how='left')

# Rellenar posibles NaN en los scores con 0
dataset_mapeado['libertad_personal_score'] = dataset_mapeado['libertad_personal_score'].fillna(0).astype(int)
dataset_mapeado['libertad_economica_score'] = dataset_mapeado['libertad_economica_score'].fillna(0).astype(int)

In [113]:
# Eliminar la columna 'Código' que ya no es necesaria
dataset_mapeado = dataset_mapeado.drop(columns=['Código'])

In [114]:
dataset_mapeado.head()

,Text,Code,libertad_personal_score,libertad_economica_score
0,DECLARACIÓN DE PRINCIPIOSFRENTE PARA LA VICTORIA,NaN,0,0
1,"Argentina, convicción y capacidad para constru...",NaN,0,0
2,La crisis ha sido el rostro de nuestro país du...,000,0,0
3,"Crisis económica, social, institucional, polít...",000,0,0
4,Sin reglas de juego claras y justas en lo econ...,503,1,-1


In [115]:
indexPrueba = 342101
print(dataset_mapeado['Text'][indexPrueba])
print(dataset_mapeado['Code'][indexPrueba],
dataset_mapeado['libertad_economica_score'][indexPrueba],
dataset_mapeado['libertad_personal_score'][indexPrueba])


El comercio se remonta a las etapas más antiguas de la Historia, y la creación del dinero ha servido para desarrollarlo y potenciarlo.
412 -1 0


In [116]:
dataset_mapeado.to_csv("Manifesto_corpus_estructurado.csv",index=False)

In [117]:
len(dataset_mapeado)

448097

In [118]:
print("Libertad Económica Score:")
print(dataset_mapeado['libertad_economica_score'].value_counts())

Libertad Económica Score:
libertad_economica_score
 0    272578
-1     94401
 1     81118
Name: count, dtype: int64


In [120]:
print("Libertad Personal Score:")
print(dataset_mapeado['libertad_personal_score'].value_counts())

Libertad Personal Score:
libertad_personal_score
 0    346331
-1     58982
 1     42784
Name: count, dtype: int64
